In [3]:
import numpy as np 
import sys
import os
from mendeleev import element
from tqdm.notebook import tqdm 
from sklearn.utils.extmath import randomized_svd

import seaborn as sns
sns.set()

from Bio.PDB import *

import warnings
warnings.filterwarnings("ignore")

In [30]:
dir_wt = './PDBs'
dir_mut = './database'

In [22]:
files_wt = os.listdir(dir_wt)
files_mut = os.listdir(dir_mut)

files_pdb = list(filter(lambda x: x.endswith('.pdb'), files_wt))
files_mut = list(filter(lambda x: x.endswith('.pdb'), files_mut))

In [31]:
files_wt

['._4FTV.pdb',
 '1XGQ.mapping',
 '2QJA.mapping',
 '2HLE.pdb',
 '._4L3E.pdb',
 '1KIQ.mapping',
 '2OOB.mapping',
 '._1SBB.pdb',
 '._2SIC.mapping',
 '._1KBH.pdb',
 '._2AK4.pdb',
 '1BD2.mapping',
 '1CSO.mapping',
 '4GNK.pdb',
 '._5E6P.mapping',
 '._4K71.pdb',
 '4NM8.mapping',
 '2B11.pdb',
 '2WPT.mapping',
 '._4JPK.pdb',
 '4NKQ.mapping',
 '._2CCL.mapping',
 '2AW2.pdb',
 '3N4I.pdb',
 '._4P23.pdb',
 '3HH2.pdb',
 '._2NU2.mapping',
 '._4KRL.mapping',
 '3HG1.mapping',
 '4KRO.pdb',
 '1QSF.pdb',
 '2PCC.mapping',
 '1MI5.pdb',
 '._3BP8.pdb',
 '1JTD.pdb',
 '3EQS.mapping',
 '1XGQ.pdb',
 '._1U7F.mapping',
 '._1UUZ.mapping',
 '4UYP.pdb',
 '2JCC.pdb',
 '._3PWP.mapping',
 '2REX.mapping',
 '3UIG.mapping',
 '._2HRK.pdb',
 '1B3S.mapping',
 '2C0L.pdb',
 '._4NM8.pdb',
 '3BN9.pdb',
 '3BTQ.mapping',
 '2VLN.mapping',
 '._1FC2.pdb',
 '2BNQ.mapping',
 '._4U6H.pdb',
 '._1TM7.mapping',
 '._1BJ1.pdb',
 '2BNQ.pdb',
 '._4GXU.pdb',
 '2A9K.pdb',
 '1AHW.mapping',
 '4G2V.mapping',
 '._1NCA.mapping',
 '._1JCK.mapping',
 '._4

In [32]:
name = files_pdb[0][:-4]
print(name)
name_ = name + '_'
if name_[0] == '.':
    name_ = name_[2:]
print(name_)

files_pdb_mut = list(filter(lambda x: x.startswith(name_), files_mut))
print(files_pdb_mut)

4FTV
4FTV_
['4FTV_DA66.pdb', '4FTV_DA93AA65.pdb', '4FTV_EA98EE101.pdb', '4FTV_AA65.pdb', '4FTV_EG99EG100ER101.pdb', '4FTV_EA98EG100ER101.pdb', '4FTV_AQ63DA93.pdb', '4FTV_EE101.pdb', '4FTV_ER101.pdb', '4FTV_AA163.pdb', '4FTV_AQ63AA66.pdb', '4FTV_DA93.pdb', '4FTV_DA66AA163.pdb', '4FTV_AQ63DA93AA66.pdb', '4FTV_EA98EG99EG100ER101.pdb']


In [33]:
files_pdb = files_pdb[:3]

In [34]:
for i in range(len(files_pdb)):
    if files_pdb[i][0] == '.':
        files_pdb[i] = files_pdb[i][2:]
print(files_pdb)

['4FTV.pdb', '2HLE.pdb', '4L3E.pdb']


In [36]:
element_list = ["C", "N", "O", "H", "S"]

def get_charge(atom):
    
    a_name = str(atom.get_name())
    if (a_name[0] in element_list and a_name != 'CL'):
        at = element(a_name[0])
    else:
        at = element(a_name)
        
    return at.atomic_number


parser = PDBParser()

eps = 1e-6
M_list_all = []
D_list_all = []

pdb_list = []

for pdb_file in files_pdb:

    # Find target protein
    name = pdb_file[:-4]

    # Get structure 
    structure = parser.get_structure(name, pdb_file)

    name_ = name + '_'
    
    if name_[0] == '.':
        name_ = name_[2:]
    
    files_pdb_mut = list(filter(lambda x: x.startswith(name_), files_mut))

    # Matrices history
    M_list = []

    # Disnances history
    D_list = []

    for file in tqdm(files_pdb_mut):
        
        pdb_list.append(files_pdb_mut)
        
        # Initialize matrix
        M = np.zeros((10000, 10000))
    
        # Get structure 
        structure_mut = parser.get_structure(name, file)

        i = 0

        for model in structure:
            for chain_idx, chain in enumerate(model):
                for residue_idx, residue in enumerate(chain):
                    for atom_idx, atom in enumerate(residue):
                
                        j = 0
                        for model_mut in structure_mut:
                            for chain_mut_idx, chain_mut in enumerate(model_mut):
                        
                                if (chain_idx == chain_mut_idx):
                                    for residue_mut_idx, residue_mut in enumerate(chain_mut):
                            
                                        if (np.abs(residue_idx - residue_mut_idx) <= 1):
                                            for atom_mut_idx, atom_mut in enumerate(residue_mut):
                                    
                                                charge_i = get_charge(atom)
                                                charge_j = get_charge(atom_mut)  
                            
                                                if (i == j or np.linalg.norm(atom.get_coord() - atom_mut.get_coord()) < eps): 
                                                    M[i, j] = 0.5 * (charge_i * charge_j)**1.2
                                                else:
                                                    M[i, j] = (charge_i * charge_j) / np.linalg.norm(atom.get_coord() - atom_mut.get_coord())
                                    
                                                j += 1
                            
                        i += 1
    
        _, S, _ = randomized_svd(M, n_components = 150)
        D_list.append(np.linalg.norm(S))
        M_list.append(M)
        
    M_list_all.append(M_list)
    D_list_all.append(D_list)
        

In [ ]:
Coulumb_data = pd.DataFrame({'#Pdb':pdb_list, 'Distance':dist_list}) #
Coulumb_data.to_csv("Distances.csv", index=False)